<a href="https://colab.research.google.com/github/pavankumarallu/Search-Engine-Models/blob/main/AutoEncoder_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, LeakyReLU, BatchNormalization, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

import numpy as np
from PIL import Image
import os 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.client import device_lib

from sklearn.cluster import KMeans
from skimage.color import rgb2hed
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics
import cv2
import pickle
from keras.models import Sequential

In [2]:
trainData = "/content/drive/MyDrive/img"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
new_train = []
file_name = []

for filename in os.listdir(trainData):

    if filename.endswith('.jpg'):
        print(filename)
        file_name.append(filename)
        image = cv2.imread(os.path.join(trainData, filename), cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image, (128, 128)) 
        new_train.append(np.asarray( resized_image, dtype="uint8" ))

# open_file = open("/content/drive/MyDrive/Data_new.pkl", "rb")
# new_train = pickle.load(open_file)
# open_file.close()


976.jpg
978.jpg
979.jpg
981.jpg
980.jpg
982.jpg
983.jpg
984.jpg
985.jpg
986.jpg
988.jpg
987.jpg
990.jpg
991.jpg
989.jpg
992.jpg
993.jpg
994.jpg
995.jpg
996.jpg
997.jpg
999.jpg
998.jpg
1002.jpg
1000.jpg
1001.jpg
1003.jpg
1004.jpg
1005.jpg
1006.jpg
1007.jpg
1008.jpg
1009.jpg
1010.jpg
1012.jpg
1011.jpg
1014.jpg
1013.jpg
1016.jpg
1015.jpg
1017.jpg
1019.jpg
1018.jpg
1020.jpg
1021.jpg
1023.jpg
1022.jpg
1026.jpg
1024.jpg
1025.jpg
1028.jpg
1027.jpg
1029.jpg
1030.jpg
1032.jpg
1031.jpg
1034.jpg
1033.jpg
1037.jpg
1035.jpg
1036.jpg
1038.jpg
1039.jpg
1040.jpg
1041.jpg
1042.jpg
1043.jpg
1044.jpg
1045.jpg
1046.jpg
1048.jpg
1047.jpg
1049.jpg
1050.jpg
1052.jpg
1051.jpg
1053.jpg
1054.jpg
1055.jpg
1056.jpg
1057.jpg
1058.jpg
1059.jpg
1061.jpg
1060.jpg
1062.jpg
1064.jpg
1063.jpg
1065.jpg
1066.jpg
1068.jpg
1069.jpg
1067.jpg
1071.jpg
1070.jpg
1072.jpg
1074.jpg
1073.jpg
1076.jpg
1075.jpg
1077.jpg
1078.jpg
1079.jpg
1081.jpg
1080.jpg
1082.jpg
1083.jpg
1084.jpg
1085.jpg
1086.jpg
1087.jpg
1090.jpg
1088.jpg
1089.j

In [5]:
file_name

['976.jpg',
 '978.jpg',
 '979.jpg',
 '981.jpg',
 '980.jpg',
 '982.jpg',
 '983.jpg',
 '984.jpg',
 '985.jpg',
 '986.jpg',
 '988.jpg',
 '987.jpg',
 '990.jpg',
 '991.jpg',
 '989.jpg',
 '992.jpg',
 '993.jpg',
 '994.jpg',
 '995.jpg',
 '996.jpg',
 '997.jpg',
 '999.jpg',
 '998.jpg',
 '1002.jpg',
 '1000.jpg',
 '1001.jpg',
 '1003.jpg',
 '1004.jpg',
 '1005.jpg',
 '1006.jpg',
 '1007.jpg',
 '1008.jpg',
 '1009.jpg',
 '1010.jpg',
 '1012.jpg',
 '1011.jpg',
 '1014.jpg',
 '1013.jpg',
 '1016.jpg',
 '1015.jpg',
 '1017.jpg',
 '1019.jpg',
 '1018.jpg',
 '1020.jpg',
 '1021.jpg',
 '1023.jpg',
 '1022.jpg',
 '1026.jpg',
 '1024.jpg',
 '1025.jpg',
 '1028.jpg',
 '1027.jpg',
 '1029.jpg',
 '1030.jpg',
 '1032.jpg',
 '1031.jpg',
 '1034.jpg',
 '1033.jpg',
 '1037.jpg',
 '1035.jpg',
 '1036.jpg',
 '1038.jpg',
 '1039.jpg',
 '1040.jpg',
 '1041.jpg',
 '1042.jpg',
 '1043.jpg',
 '1044.jpg',
 '1045.jpg',
 '1046.jpg',
 '1048.jpg',
 '1047.jpg',
 '1049.jpg',
 '1050.jpg',
 '1052.jpg',
 '1051.jpg',
 '1053.jpg',
 '1054.jpg',
 '1055.jp

In [4]:
x_train = np.asarray(new_train)

x_train = x_train.astype('float32') / 255.

In [ ]:
print(x_train.shape)


(1227, 128, 128, 3)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
generated_data = ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, rotation_range=0,  width_shift_range=0.1, height_shift_range=0.1, horizontal_flip = True, vertical_flip = False)
generated_data.fit(x_train)

In [6]:

file_name1 = "/content/drive/MyDrive/Data_new.pkl"
file_name2 = "/content/drive/MyDrive/names_new.pkl"

open_file = open(file_name1, "wb")
pickle.dump(new_train, open_file)
open_file.close()

open_file = open(file_name2, "wb")
pickle.dump(file_name, open_file)
open_file.close()

In [ ]:
n_epochs = 100
batch_size = 16
optimizer = 'adam'
loss = 'mean_squared_error'

feature_model = Sequential()
feature_model.add(Input(shape=(128, 128, 3)))
feature_model.add(Conv2D(filters=16, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(Conv2D(filters=32, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())
feature_model.add(Conv2D(filters=64, kernel_size=(3,3),strides=2, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())
feature_model.add(MaxPooling2D())
feature_model.add(Conv2D(filters=128, kernel_size=(3,3),strides=2, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())
feature_model.add(MaxPooling2D())
feature_model.add(Conv2D(filters=64, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())
feature_model.add(Conv2D(filters=32, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())

#Dense Layers
feature_model.add(tf.keras.layers.Dense(64, activation='relu'))
feature_model.add(tf.keras.layers.Dense(64, activation='relu'))
feature_model.add(tf.keras.layers.Dense(64, activation='relu'))
feature_model.add(tf.keras.layers.Dense(64, activation='relu'))
feature_model.add(tf.keras.layers.Dense(64, activation='relu'))
feature_model.add(tf.keras.layers.Dense(64, activation='relu'))
feature_model.add(tf.keras.layers.Dense(32, activation='relu'))
feature_model.add(tf.keras.layers.Dense(32, activation='relu'))
feature_model.add(tf.keras.layers.Dense(32, activation='relu'))
feature_model.add(tf.keras.layers.Dense(32, activation='relu'))
feature_model.add(tf.keras.layers.Dense(32, activation='relu'))
feature_model.add(tf.keras.layers.Dense(32, activation='relu'))
feature_model.add(tf.keras.layers.Dense(16, activation='relu'))
feature_model.add(tf.keras.layers.Dense(16, activation='relu'))
feature_model.add(tf.keras.layers.Dense(16, activation='relu'))
feature_model.add(tf.keras.layers.Dense(16, activation='relu'))
feature_model.add(tf.keras.layers.Dense(16, activation='relu'))
feature_model.add(tf.keras.layers.Dense(16, activation='relu'))
feature_model.add(tf.keras.layers.Dense(16, activation='relu',name="Feature_Layer"))

#bottle Neck
feature_model.add(Conv2D(filters=16, kernel_size=(1,1),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(Conv2D(filters=32, kernel_size=(1,1),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(Conv2D(filters=64, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())


#Decoding
feature_model.add(UpSampling2D())
feature_model.add(Conv2DTranspose(filters=128, kernel_size=(3,3),strides=2, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())
feature_model.add(UpSampling2D())
feature_model.add(Conv2DTranspose(filters=64, kernel_size=(3,3),strides=2, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())
feature_model.add(Conv2D(filters=32, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(BatchNormalization())
feature_model.add(Conv2D(filters=16, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())
feature_model.add(Conv2D(filters=3, kernel_size=(3,3),strides=1, padding='same'))
feature_model.add(LeakyReLU())

feature_model.compile(optimizer=optimizer , loss=loss, metrics=['Accuracy'])

filepath = "/February_epochs{}_batch{}_{}.h5".format(n_epochs,batch_size,optimizer)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='min', restore_best_weights=True)

callbacks_list = [checkpoint, ES]

In [ ]:
feature_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      448       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 64)        0

In [ ]:
feature_train = feature_model.fit(x_train, x_train,
                                        epochs=n_epochs,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        validation_split = 0.2,
                            
                                       )

Epoch 1/100
62/62 [==============================] - 47s 185ms/step - loss: 0.1202 - Accuracy: 0.4518 - val_loss: 0.1038 - val_Accuracy: 0.5715
Epoch 2/100
62/62 [==============================] - 9s 148ms/step - loss: 0.0242 - Accuracy: 0.5955 - val_loss: 0.0493 - val_Accuracy: 0.4510
Epoch 3/100
62/62 [==============================] - 9s 149ms/step - loss: 0.0193 - Accuracy: 0.6479 - val_loss: 0.0570 - val_Accuracy: 0.2573
Epoch 4/100
62/62 [==============================] - 9s 149ms/step - loss: 0.0186 - Accuracy: 0.6758 - val_loss: 0.0831 - val_Accuracy: 0.1886
Epoch 5/100
62/62 [==============================] - 9s 148ms/step - loss: 0.0182 - Accuracy: 0.6706 - val_loss: 0.0661 - val_Accuracy: 0.2298
Epoch 6/100
62/62 [==============================] - 9s 149ms/step - loss: 0.0173 - Accuracy: 0.6896 - val_loss: 0.0615 - val_Accuracy: 0.4325
Epoch 7/100
62/62 [==============================] - 9s 149ms/step - loss: 0.0162 - Accuracy: 0.7049 - val_loss: 0.0487 - val_Accuracy: 0.463

In [ ]:
feature_model.get_layer("Feature_Layer")

In [ ]:
feature_model.save("/content/drive/MyDrive/Model_final_Dense.h5")

In [ ]:
intermediate_layer_model = tf.keras.Model(inputs=feature_model.input,
                                       outputs=feature_model.get_layer("Feature_Layer").output)
intermediate_output = intermediate_layer_model(x_train)
train_x = intermediate_output.numpy()
np.save("feature.npy",train_x)
E_train_flatten = train_x.reshape((-1, np.prod((8,8,16))))

ResourceExhaustedError: ignored

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
knn = NearestNeighbors(n_neighbors=10, metric="cosine")
knn.fit(E_train_flatten)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [ ]:
new_test = []
test_img = cv2.imread("/content/7.jpg")
resized_image = cv2.resize(test_img, (128, 128)) 
new_test.append(np.asarray( resized_image, dtype="uint8" ))
x_test = np.asarray(new_test)
x_test = np.asarray(x_test.astype('float32') / 255.)


In [ ]:
intermediate_output = intermediate_layer_model(x_test)
test_x = intermediate_output.numpy()
E_test_flatten = test_x.reshape((-1, np.prod(( 8,8,8))))

In [ ]:
_,indeces = knn.kneighbors(E_test_flatten)

In [ ]:
indeces

array([[ 221,  108,  110,  111,  116,    4,   95, 1205,  683,    1]])